In [ ]:
# Numpy
import numpy as np

# xarray
import xarray as xr

# matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML

# time
import datetime
from cftime import num2date

import cmocean  # for nice oceanographic colormaps

#JUPYTER notebook magics
%matplotlib inline 

In [ ]:
%%bash

#link to config file
# This is just for convenience
ln -sf data-nextsim-workshop2025/nextsimdg/demo-polynya/config_polynya.cfg .


In [ ]:
%%bash

# We can also run the model with the relative or full path to the config file. It doesn't have to be linked/copied here
time /home/nextsimdg/build/nextsim --config-file config_polynya.cfg

In [ ]:
# Load the NetCDF file
ds = xr.open_dataset("polynya.diagnostic.nc", group="/data")
print(ds)

# We (still) need to fetch the mask from the init file

mask = xr.open_dataset("data-nextsim-workshop2025/nextsimdg/demo-polynya/init_polynya.nc", group="/data")['mask']
land = np.where(mask == 0, 1, np.nan)  # 1 = land, nan = ocean

In [ ]:
# Create sensible dates to use and a land mask
time = ds['time']
time_vals = num2date(time.values, units='seconds since 1970-01-01', calendar='gregorian')

time_index = 5

In [ ]:
# Striding for u and v
ny, nx = ds['u'].isel(time=0).shape
Y, X = np.meshgrid(np.arange(ny), np.arange(nx), indexing='ij')

stride = 2
u = ds['u'].isel(time=time_index)[::stride, ::stride]
v = ds['v'].isel(time=time_index)[::stride, ::stride]
x = X[::stride, ::stride]
y = Y[::stride, ::stride]

In [ ]:
# NB - we need to pick the first DG component, as well as a time slice
var = ds['cice'].isel(time=time_index).isel(dg_comp=0)

plt.figure()
p = plt.pcolormesh(var, shading='auto', cmap='cmo.ice', vmin=0, vmax=1)
plt.gca().set_aspect('equal')

plt.colorbar(p, label='Sea Ice Concentration', orientation='horizontal')

plt.quiver(x, y, u, v, scale=5, color='g', width=0.002)

plt.pcolormesh(land, shading='auto', cmap='Pastel2', vmin=0, vmax=1)

plt.title(f"Polynya model at {time_vals[time_index].isoformat()}")

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
var = ds['hice'].isel(time=time_index).isel(dg_comp=0)

plt.figure()
p = plt.pcolormesh(var, shading='auto', cmap='viridis', vmin=0, vmax=0.3)
plt.gca().set_aspect('equal')

plt.colorbar(p, label='Sea Ice Thickness', orientation='horizontal')

plt.quiver(x, y, u, v, scale=5, color='w', width=0.002)

plt.pcolormesh(land, shading='auto', cmap='Pastel2', vmin=0, vmax=1)

plt.title(f"Polynya model at {time_vals[time_index].isoformat()}")

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
ds.close()